In [1]:
def tiled_image_processing(image, radius, tile_size):
    """
    Applies a maximum filter with a given radius to the image provided as numpy-array using a tile-by-tile strategy.
    The tile_size denotes the size of the tiles in X and Y.
    The resulting, re-assembled image is returned as numpy array.
    """
    import dask
    import dask.array as da

    tiles = da.from_array(image, chunks=(tile_size, tile_size))
    
    def procedure(image):
        from scipy.ndimage import maximum_filter
        return maximum_filter(image, size=radius*2+1)

    # setup a lazy result (not computed yet)
    tile_map = da.map_blocks(procedure, tiles)

    # actually apply filter
    result = tile_map.compute()

    return result

In [2]:
def check(candidate):
    import numpy as np

    image = np.asarray([
        [0,0,0,0,0,0],
        [1,0,0,1,0,0],
        [0,0,0,0,0,0],
        [0,0,2,0,0,0],
        [0,0,0,1,0,0],
        [0,0,0,0,0,0],
    ])

    # this reference image has tile-border artefacts,
    # the maximum-filter does not consider pixels beyond
    # the 3x3 border
    reference = np.asarray([
        [1,1,0,1,1,0],
        [1,1,0,1,1,0],
        [1,1,0,1,1,0],
        [0,2,2,1,1,0],
        [0,2,2,1,1,0],
        [0,0,0,1,1,0],
    ])

    result = candidate(image, 1, 3)

    assert np.array_equal(result, reference)

In [3]:
check(tiled_image_processing)